In [43]:
from seleniumbase import Driver
from selenium.webdriver.common.by import By
import pandas as pd
import time
from selenium.common.exceptions import NoSuchElementException

In [44]:
driver = Driver(uc=True)   # undetected-chromedriver

In [45]:
BASE_URL = 'https://www.amazon.eg/s?k=%D8%A7%D9%84%D8%A7%D8%AF%D9%88%D8%A7%D8%AA+%D8%A7%D9%84%D8%B5%D8%AD%D9%8A%D9%87+%D9%84%D9%84%D8%AD%D9%85%D8%A7%D9%85+%D8%AE%D9%84%D8%A7%D8%B7%D8%A7%D8%AA&language=en&crid=3DEM3JJ05MNJG&qid=1764889905&sprefix=%2Caps%2C121&xpid=yo8Y_yeo7mmj8&ref=sr_pg_2'

In [46]:
all_urls = []
driver.get(BASE_URL)


In [47]:
for i in range(1,4):
    time.sleep(3)  # allow dynamic content to load

    # get all product <a> tags
    links = driver.find_elements(By.CSS_SELECTOR, "a.a-link-normal.s-no-outline")

    for link in links:
        url = link.get_attribute("href")
        if url and "/dp/" in url:  # filter real product links
            all_urls.append(url)
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, "a.s-pagination-next")
        if "s-pagination-disabled" in next_button.get_attribute("class"):
            break  # No more pages
        next_button.click()
        time.sleep(3)  # wait for next page to load
    except NoSuchElementException:
        break  # No "Next" button found, exit loop


In [48]:
all_urls

['https://www.amazon.eg/-/en/Chrome-Kitchen-Bathroom-Mixer-Set/dp/B0FDF8FF8X/ref=sr_1_1?crid=3DEM3JJ05MNJG&dib=eyJ2IjoiMSJ9.pRGIqOtMUoOOesRAv5FK7LMHEH4RsNbtka0LXjaOOrRhQwOK1uWPWAt3aIVQJE2_zxO408OFbiQW54XMpFZu6TkRuMzKUrAaCZL6_S7_XjDIPwZQuLQotFMHPJWevN93vmKhBgzsTMqqhYcDLU1Ho9RGs_r5ebehhAhzFMXqJvk4Imn93jb6ojp_VZt1jNraXNFfTNkZGHq2rbKX_2MTRdScrk7W9CcTtg-84ckCaM-HQV72NzoQCk97ZL2J7p-zK2b_9zKKHXlvg9fNFO--DvOg17H0IRJnhv27XZifkF8.poA5lvBzr62n4Wiz9LsZDdjmhcKBwxWUEB1KW2E3kmw&dib_tag=se&keywords=%D8%A7%D9%84%D8%A7%D8%AF%D9%88%D8%A7%D8%AA+%D8%A7%D9%84%D8%B5%D8%AD%D9%8A%D9%87+%D9%84%D9%84%D8%AD%D9%85%D8%A7%D9%85+%D8%AE%D9%84%D8%A7%D8%B7%D8%A7%D8%AA&qid=1764890393&sprefix=%2Caps%2C121&sr=8-1&xpid=yo8Y_yeo7mmj8',
 'https://www.amazon.eg/-/en/%D9%85%D8%AC%D9%85%D9%88%D8%B9%D8%A9-%D8%A7%D9%84%D8%AA%D8%B1%D9%83%D9%8A-%D8%A7%D9%84%D9%85%D8%B5%D8%B1%D9%8A-%D8%A7%D9%84%D8%A7%D8%B4%D9%83%D8%A7%D9%84-%D9%88%D8%A7%D9%84%D8%A7%D9%84%D9%88%D8%A7%D9%86/dp/B0DSVWMWZZ/ref=sr_1_2?crid=3DEM3JJ05MNJG&dib=eyJ2IjoiMSJ9.p

In [49]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

def extract_amazon_reviews(url):
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/124.0.0.0 Safari/537.36"
        ),
        "Accept-Language": "en-US,en;q=0.9"
    }

    html = requests.get(url, headers=headers).text
    soup = BeautifulSoup(html, "html.parser")

    # Product name
    product_tag = soup.find("span", {"id": "productTitle"})
    product_name = product_tag.get_text(strip=True) if product_tag else "Unknown Product"

    reviews = []

    review_blocks = soup.find_all("li", {"data-hook": "review"})

    for r in review_blocks:
        review_id = r.get("id")

        # Rating: numeric only
        rating_tag = r.find("i", {"data-hook": "review-star-rating"})
        if rating_tag:
            rating_text = rating_tag.text.strip()  # e.g., "5.0 out of 5 stars"
            rating_match = re.search(r"[\d.]+", rating_text)
            rating = float(rating_match.group()) if rating_match else None
        else:
            rating = None

        # Title: remove leading rating if present
        title_tag = r.find("a", {"data-hook": "review-title"})
        if title_tag:
            title_text = title_tag.get_text(strip=True)
            title = re.sub(r"^\d+(\.\d+)? out of 5 stars", "", title_text).strip()
        else:
            title = None

        # Date: remove "Reviewed in <Country> on"
        date_tag = r.find("span", {"data-hook": "review-date"})
        if date_tag:
            date_text = date_tag.text.strip()  # e.g., "Reviewed in Egypt on 26 July 2025"
            date_match = re.search(r"on (.+)$", date_text)
            date = date_match.group(1).strip() if date_match else date_text
        else:
            date = None

        # Body
        body_tag = r.find("span", {"data-hook": "review-body"})
        if body_tag:
            original = body_tag.find("span", class_="cr-original-review-content")
            body = original.text.strip() if original else body_tag.text.strip()
        else:
            body = None

        reviews.append({
            "product": product_name,
            "review_id": review_id,
            "rating": rating,
            "title": title,
            "date": date,
            "text": body
        })

    return reviews


In [50]:
all_reviews = []
for url in all_urls:
    reviews = extract_amazon_reviews(url)
    all_reviews.extend(reviews)  # Combine all reviews in one list

# Convert to DataFrame
df_all_reviews = pd.DataFrame(all_reviews)


In [51]:
df_all_reviews.head()

,product,review_id,rating,title,date,text
0,Chrome Kitchen and Bathroom Mixer Set - Includ...,RGDAHIJ96UFS2,5.0,ممتازة رائع خامة معقولة بالنسبة للسعر,26 July 2025,خلاط رائع تقيل
1,Chrome Kitchen and Bathroom Mixer Set - Includ...,R1OZNWYVX8RDGX,1.0,سيء,24 July 2025,منتج سيء بكل ما تحتويه الكلمة من معنيالخلاط نف...
2,مجموعة من اطقم خلاط مياه التركي و المصري علي ا...,R1RCXQK6PEGEQA,5.0,ممتاز,29 July 2025,ممتاز جدا وخامة جيدة جدا
3,مجموعة من اطقم خلاط مياه التركي و المصري علي ا...,R3EME9MVIIJGJ2,2.0,خامه اى كلام,7 August 2025,منتج اى كلام منصحش حد يشتريهالتاجر غير امين با...
4,مجموعة من اطقم خلاط مياه التركي و المصري علي ا...,R2P23SVCMINHJO,2.0,مدينه الياسمين دهشور حدائق اكتوبر,28 July 2025,مش احسن حاجة


In [52]:
df_all_reviews.duplicated().sum()

np.int64(0)

In [53]:
df_all_reviews.drop_duplicates(inplace=True)

In [54]:
df_all_reviews.shape

(159, 6)

In [55]:
df_all_reviews.to_csv("all_amazon_reviews1.csv", index=False)